In [1]:
%matplotlib inline
%run ../../import_envs.py
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 1.1.0 cuda: True


In [2]:
## Load dataset
data_path = "../gmm_dataset_c20k"
Data = torch.from_numpy(np.load(data_path + '/obs.npy')).float()

NUM_DATASETS, N, D = Data.shape
K = 3 ## number of clusters
SAMPLE_SIZE = 10
NUM_HIDDEN_LOCAL = 32

MCMC_SIZE = 10
BATCH_SIZE = 20
NUM_EPOCHS = 350
LEARNING_RATE = 1e-4
CUDA = torch.cuda.is_available()
PATH = 'soft-os-wsym'
DEVICE = torch.device('cuda:1')

Train_Params = (NUM_EPOCHS, NUM_DATASETS, SAMPLE_SIZE, BATCH_SIZE, CUDA, DEVICE, PATH)
Model_Params = (N, K, D, MCMC_SIZE)

In [3]:
from local_enc import *
from global_oneshot import *
from global_enc_v1 import *
## if reparameterize continuous variables
Reparameterized = False
# initialization
enc_z = Enc_z(K, D, NUM_HIDDEN_LOCAL, CUDA, DEVICE)
enc_eta = Enc_eta(K, D, CUDA, DEVICE, Reparameterized)
oneshot_eta = Oneshot_eta(K, D, CUDA, DEVICE, Reparameterized)

if CUDA:
    enc_z.cuda().to(DEVICE)
    enc_eta.cuda().to(DEVICE)
    oneshot_eta.cuda().to(DEVICE)
optimizer =  torch.optim.Adam(list(oneshot_eta.parameters())+list(enc_eta.parameters())+list(enc_z.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99))
models = (oneshot_eta, enc_eta, enc_z)

In [4]:
from ag_ep import *
train(models, EUBO_init_eta, optimizer, Data, Model_Params, Train_Params)

epoch: 0\250 (120s),  symKL_DB_eta: 231.073,  symKL_DB_z: 310.175,  loss: -1144.025,  ess: 3.140,  kl_eta_ex: 672.911,  kl_eta_in: 140.910,  kl_z_ex: 225.232,  kl_z_in: 34.264
epoch: 1\250 (120s),  symKL_DB_eta: 155.755,  symKL_DB_z: 171.267,  loss: -963.741,  ess: 3.578,  kl_eta_ex: 400.516,  kl_eta_in: 125.240,  kl_z_ex: 113.684,  kl_z_in: 23.747
epoch: 2\250 (120s),  symKL_DB_eta: 110.807,  symKL_DB_z: 104.837,  loss: -864.595,  ess: 3.956,  kl_eta_ex: 263.550,  kl_eta_in: 122.356,  kl_z_ex: 62.830,  kl_z_in: 17.042
epoch: 3\250 (121s),  symKL_DB_eta: 85.160,  symKL_DB_z: 71.494,  loss: -806.020,  ess: 4.270,  kl_eta_ex: 192.330,  kl_eta_in: 128.657,  kl_z_ex: 39.370,  kl_z_in: 12.757
epoch: 4\250 (121s),  symKL_DB_eta: 71.743,  symKL_DB_z: 54.909,  loss: -767.017,  ess: 4.510,  kl_eta_ex: 155.300,  kl_eta_in: 143.381,  kl_z_ex: 28.040,  kl_z_in: 10.012
epoch: 5\250 (120s),  symKL_DB_eta: 65.058,  symKL_DB_z: 45.868,  loss: -747.593,  ess: 4.673,  kl_eta_ex: 139.633,  kl_eta_in: 163

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/hao/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3296, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-d8a561a5e9a6>", line 2, in <module>
    train(models, EUBO_init_eta, optimizer, Data, Model_Params, Train_Params)
  File "../training.py", line 29, in train
    loss, metric_step, reused = objective(models, obs, SubTrain_Params)
  File "../objectives/ag_ep.py", line 39, in EUBO_init_eta
    q_eta, p_eta, q_nu = enc_eta(obs, state, K, D)
  File "/home/hao/anaconda3/lib/python3.7/site-packages/torch/nn/modules/module.py", line 493, in __call__
    result = self.forward(*input, **kwargs)
  File "../models/global_enc_v1.py", line 65, in forward
    name='means')
  File "<string>", line 2, in f
  File "/home/hao/Research/probtorch/probtorch/stochastic.py", line 287, in variable
    node = RandomVariable(dist, value, provenance, mask=self._mask)
  File "/home/hao/Rese

KeyboardInterrupt: 

In [5]:
# torch.save(enc_z.state_dict(), "../weights/enc-z-%s" % PATH)
# torch.save(enc_eta.state_dict(), "../weights/enc-eta-%s" % PATH)
# torch.save(oneshot_eta.state_dict(), "../weights/oneshot-eta-%s" % PATH)